In [1]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
from typing import Literal

In [2]:
@tool()
def write_file(file_name: str, content: str) -> str:
    """Write content to a file"""
    with open(file_name, "w") as f:
        f.write(content)
    return f"Successfully wrote for file '{file_name}'"

@tool()
def read_file(file_name: str) -> str:
    """Read content of a file"""
    try:
        with open(file_name, "r") as f:
            return f.read()
    except FileNotFoundError:
        return f"File {file_name} not found"

In [3]:
agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[write_file, read_file],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "write_file": True,
                "read_file": False
            },
            description_prefix="Action requires your approval"
        )
    ],
    checkpointer=InMemorySaver()
)

In [4]:
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": {"role":"user", "content": "Write 'Hi guys, welcome to my YouTube channel' to a file welcome.txt"}},
    config=config
)

response

{'messages': [HumanMessage(content="Write 'Hi guys, welcome to my YouTube channel' to a file welcome.txt", additional_kwargs={}, response_metadata={}, id='cf4e158a-966c-41eb-9ec2-c7519b5c52da'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'write_file', 'arguments': '{"file_name": "welcome.txt", "content": "Hi guys, welcome to my YouTube channel"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--cae30051-f3ec-4718-ade7-6b9b620ff518-0', tool_calls=[{'name': 'write_file', 'args': {'file_name': 'welcome.txt', 'content': 'Hi guys, welcome to my YouTube channel'}, 'id': '0b1c775c-55b7-46dc-9d45-808159246677', 'type': 'tool_call'}], usage_metadata={'input_tokens': 111, 'output_tokens': 31, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}})],
 '__interrupt__': [Interrupt(value={'action_requ

In [5]:
if "__interrupt__" in response:
    interrupt_data = response["__interrupt__"][0].value

In [6]:
interrupt_data

{'action_requests': [{'name': 'write_file',
   'args': {'file_name': 'welcome.txt',
    'content': 'Hi guys, welcome to my YouTube channel'},
   'description': "Action requires your approval\n\nTool: write_file\nArgs: {'file_name': 'welcome.txt', 'content': 'Hi guys, welcome to my YouTube channel'}"}],
 'review_configs': [{'action_name': 'write_file',
   'allowed_decisions': ['approve', 'edit', 'reject']}]}

In [7]:
print(interrupt_data["action_requests"][0]["description"])

Action requires your approval

Tool: write_file
Args: {'file_name': 'welcome.txt', 'content': 'Hi guys, welcome to my YouTube channel'}


In [8]:
interrupt_data["review_configs"]

[{'action_name': 'write_file',
  'allowed_decisions': ['approve', 'edit', 'reject']}]

In [9]:
user_decision: Literal["approve", "edit", "reject"]  = input("Enter your decision (approve/edit/reject)")
user_decision

'edit'

In [ ]:
if user_decision == "approve":
    result = agent.invoke(
        Command(
            resume={
                "decisions":[
                    { "type": "approve" }
                ]
            }
        ), 
        config=config
    )


elif user_decision == "reject":
    result = agent.invoke(
        Command(
            resume={
                "decisions":[
                    { "type": "reject" } 
                ]
            }), 
        config=config
    )

    
elif user_decision == "edit":
    new_content = input("Type new content for the file.")
    result = agent.invoke(
        Command(
            resume={
                "decisions": [
                    {
                        "type": "edit",
                        "edited_action": {
                            "name": "write_file",
                            "args": {
                                "file_name": "welcome.txt", 
                                "content": new_content
                            },
                        }
                    }
                ]
            }
        ),
        config=config  
)
    

In [11]:
print(result["messages"][-2].content)

Successfully wrote for file 'welcome.txt'
